In [1]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-01",#2023-07-01-preview",#2024-02-15-preview",#
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

## 1. Test Function Calling

In [2]:
def get_function_call(messages, function_call = "auto"):
    # Define the functions to use
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=messages,
    functions=functions,
    function_call=function_call)


    return response.model_dump_json(indent=2)

In [3]:
first_message = [{"role": "user", "content": "What's the weather like in San Francisco?"}]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
print (get_function_call(first_message, "auto"))

# 'none' : Don't call any function 
print("Don't call any function:")
print (get_function_call(first_message, "none"))

# force a specific function call
print("Force a specific function call:")
print (get_function_call(first_message, function_call={"name": "get_current_weather"}))

Let the model decide what function to call:
{
  "id": "chatcmpl-9h6MdXKhncBHaOg02i7iallCOQJFv",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\"location\":\"San Francisco, CA\",\"unit\":\"fahrenheit\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      },
      "content_filter_results": {}
    }
  ],
  "created": 1720059595,
  "model": "gpt-4",
  "object": "chat.completion",
  "system_fingerprint": "fp_811936bd4f",
  "usage": {
    "completion_tokens": 23,
    "prompt_tokens": 83,
    "total_tokens": 106
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
      

## 2. Define Functions

In [4]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [5]:
get_current_time("America/New_York")

'10:20:01 PM'

## 3. Call a Function using AOAI

### 3.1 Describe the functions so that the model knows how to call them

In [6]:
functions = [
        {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
                "required": ["location"],
            },
        }
    ]

available_functions = {
            "get_current_time": get_current_time
        } 

### 3.2 Define a helper function to validate the function call

In [7]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [8]:
import json

In [9]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=first_message,
    functions=functions,
    function_call="auto")


response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content="I'm sorry, but I'm currently unable to provide real-time weather updates. To get the most accurate and up-to-date weather information for San Francisco, I recommend checking a reliable weather forecasting website or app like the National Weather Service, The Weather Channel, or AccuWeather.", role='assistant', function_call=None, tool_calls=None)


In [10]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=messages,
    functions=functions,
    function_call="auto")


    response_message = response.choices[0].message


    # Step 2: check if GPT wanted to call a function
    if response_message.function_call:
        print("Recommended Function call:")
        print(response_message.function_call)
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message.function_call.name
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message.function_call.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "name": response_message.function_call.name,
                "content": response_message.function_call.arguments,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        # get a new response from GPT where it can see the function response
        second_response = client.chat.completions.create(
        model="gpt-4-0125-Preview", # model = "deployment_name".
        messages=messages,
        )
        return second_response.choices[0].message.content

In [11]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, functions, available_functions, "gpt-4-32k")
print(assistant_response)

Recommended Function call:
FunctionCall(arguments='{"location":"America/New_York"}', name='get_current_time')

Output of function call:
10:20:26 PM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'name': 'get_current_time', 'content': '{"location":"America/New_York"}'}
{'role': 'function', 'name': 'get_current_time', 'content': '10:20:26 PM'}

The current time in New York is 10:20 PM.
